In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
import pandas as pd
import datetime

import src.data_cleaning as dc

In [4]:
# defining the 4 dataframes from our .py file

df_sales, df_resbldg, df_parcel, df_lookup = dc.create_dfs('EXTR_RPSale.csv', 
                                                           'EXTR_ResBldg.csv', 
                                                           'EXTR_Parcel.csv',
                                                           'EXTR_LookUp.csv')

/Users/kyledecember1/opt/anaconda3/envs/kh-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/kyledecember1/opt/anaconda3/envs/kh-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
# filter for sales from 2019

df_sales_2019 = dc.sales_2019(df_sales)

In [11]:
# combine major + minor to create HID column

df_sales_2019['HID'] = (df_sales_2019['Major'].astype(str)).str.zfill(6) + '-' + (df_sales_2019['Minor'].astype(str)).str.zfill(4)
df_resbldg['HID'] = (df_resbldg['Major'].astype(str)).str.zfill(6) + '-' + (df_resbldg['Minor'].astype(str)).str.zfill(4)
df_parcel['HID'] = (df_parcel['Major'].astype(str)).str.zfill(6) + '-' + (df_parcel['Minor'].astype(str)).str.zfill(4)

<ipython-input-11-447d97aa6cac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales_2019['HID'] = (df_sales_2019['Major'].astype(str)).str.zfill(6) + '-' + (df_sales_2019['Minor'].astype(str)).str.zfill(4)


In [12]:
#retain only relevant columns for df_sales_2019

columns = ['Major', 'Minor', 'DocumentDate', 'SalePrice', 'PropertyType', 'HID']
df_sales_2019 = df_sales_2019.loc[:, columns]

In [13]:
# new column for whether or not the location is waterfront property

df_parcel['waterfront'] = df_parcel['WfntLocation'] > 0

In [18]:
df_parcel['AirportNoise'].unique()

array([ 0, 67, 59, 60, 66, 63, 61, 64, 65, 70, 62, 69, 68,  1, 50])

In [27]:
# new column for whether or not there are power lines

df_parcel['pwrlines'] = df_parcel['PowerLines'] == 'Y'

# new column for whether or not there is some other nusiance

df_parcel['othernuisance'] = df_parcel['OtherNuisances'] == 'Y'

In [36]:
# new column summing integer values corresponding to nuisances **pwrlines and othernuisance will be 1 for true
# and 0 for false
df_parcel['nuisance_total'] = (
    df_parcel['AirportNoise'] +
    df_parcel['TrafficNoise'] +
    df_parcel['pwrlines'] +
    df_parcel['othernuisance']
)

In [65]:
df_parcel['nuisance_bool'] = df_parcel['nuisance_total'] > 0

In [66]:
df_parcel['nuisance_bool']

0         False
1         False
2         False
3         False
4         False
          ...  
614332     True
614333    False
614334    False
614335    False
614336    False
Name: nuisance_bool, Length: 614337, dtype: bool

In [67]:
df_combined = df_sales_2019.merge(df_parcel, how='inner', on='HID')
df_combined = df_combined.merge(df_resbldg, how='inner', on='HID')

In [68]:
df_combined

,Major_x,Minor_x,DocumentDate,SalePrice,PropertyType,HID,Major_y,Minor_y,PropName,PlatName,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,213043,120,12/20/2019,560000,11,213043-0120,213043,120,,DUVALL HILLS DIV NO. 04,...,0,0,0,1989,0,0,0,0,3,0
1,940652,630,07/22/2019,435000,11,940652-0630,940652,630,,WILDERNESS ESTATES,...,1,0,0,1994,0,0,0,0,3,2500
2,797320,2320,03/27/2019,540000,3,797320-2320,797320,2320,,STATE ADD TO SEATTLE NO. 05,...,0,0,0,1930,0,0,0,0,5,0
3,797320,2320,03/28/2019,0,3,797320-2320,797320,2320,,STATE ADD TO SEATTLE NO. 05,...,0,0,0,1930,0,0,0,0,5,0
4,663990,178,10/02/2019,0,3,663990-0178,663990,178,,PARISHS GARDEN TRS,...,1,0,0,1994,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43540,231430,740,09/16/2019,700988,11,231430-0740,231430,740,,ELLIOTT FARM,...,1,0,0,1998,0,0,0,0,3,4500
43541,192300,170,09/03/2019,1550000,11,192300-0170,192300,170,,DAWN VISTA,...,1,0,0,1999,0,0,5,0,3,0
43542,757820,400,07/19/2019,930000,11,757820-0400,757820,400,,SCENIC HEIGHTS ADD SUPL TO W S,...,1,0,1,1948,0,0,0,0,5,0
43543,143800,420,07/09/2019,695000,11,143800-0420,143800,420,,CAVALLA,...,0,0,0,2014,0,0,0,0,3,6000


In [78]:
df_resbldg.columns

Index(['Major', 'Minor', 'BldgNbr', 'NbrLivingUnits', 'Address',
       'BuildingNumber', 'Fraction', 'DirectionPrefix', 'StreetName',
       'StreetType', 'DirectionSuffix', 'ZipCode', 'Stories', 'BldgGrade',
       'BldgGradeVar', 'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor',
       'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving',
       'SqFtTotBasement', 'SqFtFinBasement', 'FinBasementGrade',
       'SqFtGarageBasement', 'SqFtGarageAttached', 'DaylightBasement',
       'SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem',
       'HeatSource', 'BrickStone', 'ViewUtilization', 'Bedrooms',
       'BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'FpSingleStory',
       'FpMultiStory', 'FpFreestanding', 'FpAdditional', 'YrBuilt',
       'YrRenovated', 'PcntComplete', 'Obsolescence', 'PcntNetCondition',
       'Condition', 'AddnlCost', 'HID'],
      dtype='object')

In [100]:
columns = ['SalePrice', 'SqFtTotLiving', 'NbrLivingUnits', 'Stories']
df = df_combined[columns]
df.corr()

,SalePrice,SqFtTotLiving,NbrLivingUnits,Stories
SalePrice,1.000000,0.196273,0.120685,0.115316
SqFtTotLiving,0.196273,1.000000,0.094034,0.371277
NbrLivingUnits,0.120685,0.094034,1.000000,0.014060
Stories,0.115316,0.371277,0.014060,1.000000


In [98]:
columns = df_combined.columns
df = df_combined[columns]
df.corr()['SalePrice'].sort_values(ascending=False)[0:20]

SalePrice             1.000000
BldgGrade             0.205253
SqFtTotLiving         0.196273
SqFt2ndFloor          0.156676
BathFullCount         0.132804
SqFt1stFloor          0.120696
NbrLivingUnits        0.120685
LakeWashington        0.117461
Stories               0.115316
SqFtOpenPorch         0.109235
Bedrooms              0.099272
HBUAsIfVacant         0.099224
SqFtGarageAttached    0.091971
TidelandShoreland     0.086937
Territorial           0.078781
YrBuilt               0.077504
BathHalfCount         0.077479
Bath3qtrCount         0.072025
Township              0.067538
waterfront            0.065549
Name: SalePrice, dtype: float64